In [1]:
import pandas as pd
import numpy as np
import time

In [2]:
ratings = pd.read_csv('game_ratings.csv.gz', encoding='utf-8', compression='gzip', index_col=0)

In [3]:
ratings.shape

(290, 63110)

In [4]:
ratings.drop(ratings.columns[ ratings.notnull().sum() < 3 ], axis=1, inplace=True)

In [5]:
ratings.shape

(290, 25048)

In [6]:
ratings.head()


,AceAceBaby,AdmiralACF,agreen53,ALSB,Altasia,Altoid Guy,anatana,AnotherHorrorFan,Armadillo Al,auroramu,...,Wire Bender,yams69,yggsdrasil23,Zaksgirl,chutspe,boywonder_uk,gfeiden,Mikis,MrCryptic,MrFlibble2999
175223,8.0,8.0,8.0,7.0,8.0,8.0,9.0,8.5,7.0,6.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40834,NaN,NaN,NaN,NaN,NaN,8.0,NaN,9.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178900,NaN,5.0,NaN,NaN,NaN,NaN,9.0,9.0,9.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104162,NaN,6.0,6.0,NaN,NaN,NaN,NaN,8.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36218,5.0,NaN,3.0,NaN,NaN,8.0,NaN,9.0,9.0,7.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
mean = ratings.mean()
pd.DataFrame(mean).head()

,0
AceAceBaby,6.796296
AdmiralACF,6.257143
agreen53,6.486486
ALSB,7.500000
Altasia,7.100000


In [9]:
normalized = ratings.sub(mean, axis=1)
normalized.head()

,AceAceBaby,AdmiralACF,agreen53,ALSB,Altasia,Altoid Guy,anatana,AnotherHorrorFan,Armadillo Al,auroramu,...,Wire Bender,yams69,yggsdrasil23,Zaksgirl,chutspe,boywonder_uk,gfeiden,Mikis,MrCryptic,MrFlibble2999
175223,1.203704,1.742857,1.513514,-0.5,0.9,0.259259,1.146154,0.408537,-0.051282,-0.428571,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40834,NaN,NaN,NaN,NaN,NaN,0.259259,NaN,0.908537,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
178900,NaN,-1.257143,NaN,NaN,NaN,NaN,1.146154,0.908537,1.948718,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104162,NaN,-0.257143,-0.486486,NaN,NaN,NaN,NaN,-0.091463,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
36218,-1.796296,NaN,-3.486486,NaN,NaN,0.259259,NaN,0.908537,1.948718,0.071429,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
sqrt_sum_squares = normalized.pow(2).sum(axis=1).pow(0.5)


0.0909740924835


In [13]:
import sys

sim = pd.DataFrame()

def similarity(i, j):
    nom = (normalized.loc[i]*normalized.loc[j]).sum()
    denom = sqrt_sum_squares.loc[i]*sqrt_sum_squares.loc[j]
    return nom/denom

t1 = time.time()
sys.stdout.write('=' * normalized.index.shape[0])
sys.stdout.write('\n')
for i_index, i in enumerate(normalized.index.values):
    for j_index, j in enumerate(normalized.index.values):
        if (i_index <= j_index):
            sim.loc[i,j] = similarity(i,j)
        else:
            sim.loc[i,j] = sim.loc[j,i]

    sys.stdout.write('.')
    sys.stdout.flush()
print ''
t2 = time.time()
print (t2-t1)


/Users/merlyn/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:8: RuntimeWarning: invalid value encountered in double_scalars


..................................................................................................................................................................................................................................................................................................
66.9106068611


In [49]:
sim.to_csv('simularity.csv.gz', compression='gzip')

In [19]:
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

t1 = time.time()

normalized.fillna(value=0, inplace=True)
normalized_sparse = sparse.csr_matrix(normalized)
similarities = cosine_similarity(normalized_sparse)
# print('pairwise dense output:\n {}\n'.format(similarities))

t2 = time.time()
print (t2-t1)

0.198402881622
